<a href="https://colab.research.google.com/github/scarfboy/wetsuite-dev/blob/main/notebooks/extras/extras_diagnose_kamervragen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# on colab, start with
!pip --quiet install https://github.com/scarfboy/wetsuite-dev/archive/refs/heads/main.zip

# Purpose of this notebook

Inspect how many cases of question-asnwer mismatch are there in the `kamervragen` dataset.

In [ ]:
import wetsuite.datasets

kamervragen = wetsuite.datasets.load('kamervragen-struc')

In [1]:
print(kamervragen.description)

NameError: name 'kamervragen' is not defined

# How many cases are there of grouped answers leading to mismatches in numbering?

In [ ]:
mismatched_qa, matched_qa = 0,0

for id, details in list(kamervragen.data.items()):
    vraagdata = details['vraagdata']

    vraag_numbers = set()
    antwoord_numbers = set()

    for qa_number in vraagdata:
        if 'vraag' in vraagdata[qa_number]:
            vraag_numbers.add( qa_number )
        if 'antwoord' in vraagdata[qa_number]:
            antwoord_numbers.add( qa_number )

        diff = vraag_numbers.symmetric_difference( antwoord_numbers )
        if len(diff) == 0:
            matched_qa    += 1
        else:
            mismatched_qa += 1
            if 0:  # showing all cases in the dataset would be very spammy
                print('\n=== %s ==='%id)
                print("Vragen:     %s"%sorted(vraag_numbers))
                print("Antwoorden: %s"%sorted(antwoord_numbers))
                print("Difference: %s"%sorted(diff))
                print( details['urls'][-1] )  # mention the document we used, so we can look at it

print( f"Matched: {matched_qa}     Not matched: {mismatched_qa}" )

Matched: 219302     Not matched: 12259
